
## Boosting as an Ensemble Method

---
Boosting takes a weak base learner and tries to make it a strong learner by retraining it on the misclassified samples.

1) **Base model fitting is an iterative procedure**: It cannot be run in parallel.
- **Weights are assigned to observations to indicate their "importance:"** Samples with higher weights are given higher influence on the total error of the next model, prioritizing those observations.
- **Weights change at each iteration with the goal of correcting the errors/misclassifications of the previous iteration**: The first base estimator is fit with uniform weights on the observations.
- **Final prediction is typically constructed by a weighted vote**: Weights for each base model depend on their training errors or misclassification rates.

## Pros and Cons of Boosting

---

### Pros

- Achieves higher performance than bagging when the hyperparameters are properly tuned.
- Works equally well for classification and regression.
- Can use "robust" loss functions that make the model resistant to outliers.

---

### Cons

- Difficult and time consuming to properly tune hyperparameters.
- Cannot be parallelized like bagging (bad scalability when there are huge amounts of data).
- Higher risk of overfitting compared to bagging.


## Boosting and the Bias-Variance Trade-Off

---

Recall that **bagging aims to reduce variance** (reduce overfitting).

**Boosting aims to reduce bias** (and can reduce variance a bit as well)! (reduce underfitting)

### Why?

The rationale/theory behind boosting is to combine **many weak learners into a single strong learner.** Instead of using deep/full decision trees like in bagging, **boosting uses shallow/high-bias base estimators.**

Thus, each weak learner has:

- Low variance.
- High bias.

It uses iterative fitting to explain error/misclassification unexplained by the previous base models and reduces bias without increasing variance.


## AdaBoost
---

The core principle of AdaBoost is to **fit a sequence of weak learners** (i.e., models that are only slightly better than random guessing, such as a single-split tree) **on repeatedly modified versions of the data**. After each fit, the importance weights on each observation need to be updated. 

The predictions are then combined through a weighted majority vote (or sum) to produce the final prediction. AdaBoost, like all boosting ensemble methods, focuses the next model's fit on the misclassifications/weaknesses of the prior models.

All training examples start with equal importance weighting. When we finish training a classifier, we update the importance weighting of the classifier itself, represented by alpha $\alpha$:

As iterations continue, **examples that are difficult to predict receive ever-increasing influence**. Each subsequent weak learner is thereby forced to concentrate on the examples that are missed by the previous ones in the sequence.

## Gradient Boosting
---

Gradient boosting classifiers are a generalization of boosting to arbitrary, differentiable loss functions. The intuition behind this mechanism is to:

1. Fit a model $F$ to the data.
2. Look at the difference between our observed $y$ and our model $F$. (The $y_i - F(x_i)$ can be thought of as residuals!)
3. Fit a second model, $F_2$, to (roughly) the residuals $y_i - F(x_i)$.
4. Aggregate your model $F$ and $F_2$. While we won't get into the details now, we can interpret residuals as negative gradients. By doing this, we can apply our gradient descent algorithm to optimize our loss and generalize this to many loss functions.

GBRT is an accurate and effective off-the-shelf procedure that can be used for both regression and classification problems. Gradient tree boosting models are used in a variety of areas, including web search ranking and ecology.

**The advantages of GBRT are:**

- Natural handling of mixed data types (= heterogeneous features).
- Predictive power.
- Robustness to outliers in output space (via robust loss functions).

**The disadvantages of GBRT are:**
- Scalability: Due to the sequential nature of boosting, it can hardly be parallelized.
- Difficult hyperparameters to tune.

## The Difference Between the AdaBoost and Gradient Boosting
---
- AdaBoost is about reweighting the preceding model's errors in subsequent iterations.
- Gradient boosting is about fitting subsequent models to the residuals of the last model.

## Code for boosting

In [4]:
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.datasets import make_classification

In [5]:
# Load dataset from make_classificatioabsn
X, y = make_classification(n_samples=750, n_features=20, random_state=42)

In [7]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

### AdaBoost

In [8]:
ada = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())
ada_params = {
    'n_estimators': [50,100],
    'base_estimator__max_depth': [1,2],
    'learning_rate': [.9, 1.]
}
gs = GridSearchCV(ada, param_grid=ada_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

0.9216729244889446


{'base_estimator__max_depth': 2, 'learning_rate': 0.9, 'n_estimators': 100}

### Gradient Boosting

In [9]:
gboost = GradientBoostingClassifier()
gboost_params = {
    'max_depth': [2,3,4],
    'n_estimators': [100, 125, 150],
    'learning_rate': [.08, .1, .12]
}
gb_gs = GridSearchCV(gboost, param_grid=gboost_params, cv=3)
gb_gs.fit(X_train, y_train)
print(gb_gs.best_score_)
gb_gs.best_params_

0.9341601243979216


{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100}

### VotingClassifier

In [10]:
vote = VotingClassifier([
    ('tree', DecisionTreeClassifier()),
    ('ada', AdaBoostClassifier()),
    ('gb', GradientBoostingClassifier())
])
vote_params = {
    'ada__n_estimators': [50,75],
    'gb__n_estimators': [100,125],
    'tree__max_depth': [None, 5]
}
gs = GridSearchCV(vote, param_grid=vote_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

0.9305855804604241


{'ada__n_estimators': 75, 'gb__n_estimators': 100, 'tree__max_depth': 5}